In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, when, trim, split, regexp_replace, round, lower, col, encode, count, levenshtein, row_number, broadcast, coalesce

In [2]:
spark = SparkSession.builder\
    .appName("AggregateCatalogueCo2")\
    .enableHiveSupport()\
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/13 16:29:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

df_catalogue = spark.sql("SELECT * FROM catalogue_ext")
df_co2 = spark.sql("SELECT * FROM crit_air_ext")

df_catalogue = df_catalogue.filter(df_catalogue['marque'] != 'marque') # A GERER DANS L'IMPORT DE HIVE ???

In [4]:
df_co2.show()
df_catalogue.show()

+--------------------+-----------+----------+------------+
|       marque_modele|bonus_malus|rejets_co2|cout_energie|
+--------------------+-----------+----------+------------+
|      CITROEN C-ZERO|  -6 000€ 1|       0.0|       491 €|
|MERCEDES SPRINTER...|    +8 753€|     200.0|       799 €|
|VOLKSWAGEN Passat...|          -|      31.0|        56 €|
|    SMART EQ FORFOUR|  -6 000€ 1|       0.0|       175 €|
|BENTLEY BENTAYGA ...|          -|      84.0|       102 €|
|SMART EQ FORTWO C...|  -6 000€ 1|       0.0|       175 €|
|SMART EQ FORFOUR ...|  -6 000€ 1|       0.0|       213 €|
|AUDI Q5 50 TFSI e...|          -|      49.0|       105 €|
|MERCEDES SPRINTER...|    +8 753€|     255.0|       988 €|
|MERCEDES SPRINTER...|    +8 753€|     200.0|       799 €|
|KIA SOUL Moteur Ã...|  -6 000€ 1|       0.0|       214 €|
|MERCEDES VITO Tou...|  -6 000€ 1|       0.0|       411 €|
|MERCEDES SPRINTER...|    +8 753€|     262.0|       999 €|
|SMART EQ FORFOUR ...|  -6 000€ 1|       0.0|       213 

### Remarque 001

- **co2** dispose de marque et modele dans la meme colone
- La colone **nom** du catalogue n'est pas nommée **modele** dans **co2**
- La colone **modele** dans les 2 tableaux n'ont pas la meme casse.
- La colone **marque** dans les 2 tableaux n'ont pas la meme casse.
- Le signe **€** est mentionné dans la colone **bonus_malus** de co2.
- Le signe **€** est mentionné dans la colone **cout_energie** de co2.
- Le chiffre 1 peut apparaitre après le signe **€** dans la colone **bonus_malus** de co2.
- Le symbole "�" apparaît dans la colonne **longueur**.


In [5]:
# co2 dispose de marque et modele dans la meme colone
df_co2 = df_co2.withColumn("marque", split(df_co2["marque_modele"], " ", 2).getItem(0))
df_co2 = df_co2.withColumn("modele", split(df_co2["marque_modele"], " ", 2).getItem(1))
df_co2 = df_co2.drop('marque_modele')

# La colone **nom** du catalogue n'est pas nommée **modele** dans dans **co2**
df_catalogue = df_catalogue.withColumnRenamed("nom", "modele")

# La colone modele dans les 2 tableaux n’ont pas la meme casse.
df_co2 = df_co2.withColumn("marque", lower(trim(col("marque"))))
df_catalogue = df_catalogue.withColumn("marque", lower(trim(col("marque"))))

# La colone modele dans les 2 tableaux n’ont pas la meme casse.
df_co2 = df_co2.withColumn("modele", lower(trim(col("modele"))))
df_catalogue = df_catalogue.withColumn("modele", lower(trim(col("modele"))))

# Le signe € est mentionné dans la colone bonus_malus de co2.
df_co2 = df_co2.withColumn("bonus_malus", split(trim(df_co2["bonus_malus"]), "€").getItem(0))
# Le signe € est mentionné dans la colone cout_energie de co2.
df_co2 = df_co2.withColumn("cout_energie", split(trim(df_co2["cout_energie"]), "€").getItem(0))

# Le chiffre 1 peut apparaitre après le signe € dans la colone bonus_malus de co2.
df_co2 = df_co2.withColumn("bonus_malus", regexp_replace(trim(df_co2["bonus_malus"]), "[^0-9-]", "").cast("float"))

#Le symbole "�" apparaît dans la colonne longueur.
df_catalogue = df_catalogue.withColumn("longueur", regexp_replace(col("longueur"), "�", "e"))



In [6]:
df_co2.show()
df_catalogue.show()

+-----------+----------+------------+----------+--------------------+
|bonus_malus|rejets_co2|cout_energie|    marque|              modele|
+-----------+----------+------------+----------+--------------------+
|    -6000.0|       0.0|        491 |   citroen|              c-zero|
|     8753.0|     200.0|        799 |  mercedes|sprinter combi 31...|
|       null|      31.0|         56 |volkswagen|passat sw 1.4 tsi...|
|    -6000.0|       0.0|        175 |     smart|          eq forfour|
|       null|      84.0|        102 |   bentley|     bentayga hybrid|
|    -6000.0|       0.0|        175 |     smart|eq fortwo cabrio ...|
|    -6000.0|       0.0|        213 |     smart|eq forfour 7 kw m...|
|       null|      49.0|        105 |      audi|q5 50 tfsi e (299...|
|     8753.0|     255.0|        988 |  mercedes|sprinter combi 31...|
|     8753.0|     200.0|        799 |  mercedes|sprinter combi 31...|
|    -6000.0|       0.0|        214 |       kia|soul moteur ã©lec...|
|    -6000.0|       

### Remarque 002

- Recherche d'éventuel autre "�".
- Correction suivant résultat des recherches de "�"

In [7]:
#Recherche d'éventuel autre "�".
df_search_special_char_catalogue = df_catalogue.filter(
    col("marque").like("%�%") | 
    col("modele").like("%�%") | 
    col("couleur").like("%�%") | 
    col("longueur").like("%�%")
)

df_search_special_char_co2 = df_co2.filter(
    col("marque").like("%�%") | 
    col("modele").like("%�%") 
)


df_search_special_char_catalogue.show()
df_search_special_char_co2.show()


+-------+----------+---------+--------+--------+--------+-------+--------+-------+
| marque|    modele|puissance|longueur|nbplaces|nbportes|couleur|occasion|   prix|
+-------+----------+---------+--------+--------+--------+-------+--------+-------+
|hyunda�|matrix 1.6|      103|  longue|       7|       5|   bleu|   false|15960.0|
|hyunda�|matrix 1.6|      103|  longue|       7|       5|  blanc|   false|15960.0|
|hyunda�|matrix 1.6|      103|  longue|       7|       5|   gris|   false|15960.0|
|hyunda�|matrix 1.6|      103|  longue|       7|       5|   noir|   false|15960.0|
|hyunda�|matrix 1.6|      103|  longue|       7|       5|  rouge|   false|15960.0|
+-------+----------+---------+--------+--------+--------+-------+--------+-------+

+-----------+----------+------------+------+------+
|bonus_malus|rejets_co2|cout_energie|marque|modele|
+-----------+----------+------------+------+------+
+-----------+----------+------------+------+------+



In [8]:
# Étape 1: Extraire les marques correctes de df_co2
marques_correctes_df = df_co2.select('marque').distinct().alias('marques_correctes')

# Étape 2: Extraire les marques du catalogue
marques_catalogue_df = df_catalogue.select('marque').distinct().alias('marques_catalogue')

# Étape 3: Créer le mapping des marques
df_cross = marques_catalogue_df.crossJoin(broadcast(marques_correctes_df))

df_cross = df_cross.withColumn('distance', levenshtein(col('marques_catalogue.marque'), col('marques_correctes.marque')))

window = Window.partitionBy('marques_catalogue.marque').orderBy(col('distance'))
df_min_distance = df_cross.withColumn('rn', row_number().over(window)).filter(col('rn') == 1)

marque_mapping = df_min_distance.select(
    col('marques_catalogue.marque').alias('marque_catalogue'),
    col('marques_correctes.marque').alias('marque_correcte'),
    'distance'
).filter(col('distance') <= 2)

# Étape 4: Appliquer le mapping à df_catalogue
df_catalogue_corrected = df_catalogue.join(
    marque_mapping,
    df_catalogue.marque == marque_mapping.marque_catalogue,
    how='left'
)

df_catalogue_corrected = df_catalogue_corrected.withColumn(
    'marque',
    coalesce(col('marque_correcte'), col('marque'))
).drop('marque_catalogue', 'marque_correcte', 'distance')

# Vérification du résultat
df_catalogue_corrected.printSchema()

# Optionnel : Remplacer df_catalogue par le DataFrame corrigé
df_catalogue = df_catalogue_corrected

df_catalogue.show(truncate=False)

root
 |-- marque: string (nullable = true)
 |-- modele: string (nullable = true)
 |-- puissance: integer (nullable = true)
 |-- longueur: string (nullable = true)
 |-- nbplaces: integer (nullable = true)
 |-- nbportes: integer (nullable = true)
 |-- couleur: string (nullable = true)
 |-- occasion: boolean (nullable = true)
 |-- prix: float (nullable = true)

+----------+--------------+---------+-----------+--------+--------+-------+--------+-------+
|marque    |modele        |puissance|longueur   |nbplaces|nbportes|couleur|occasion|prix   |
+----------+--------------+---------+-----------+--------+--------+-------+--------+-------+
|volkswagen|touran 2.0 fsi|150      |longue     |7       |5       |rouge  |false   |27340.0|
|volkswagen|touran 2.0 fsi|150      |longue     |7       |5       |gris   |true    |19138.0|
|volkswagen|touran 2.0 fsi|150      |longue     |7       |5       |bleu   |true    |19138.0|
|volkswagen|touran 2.0 fsi|150      |longue     |7       |5       |gris   |false 

In [9]:
marques_count_catalogue = df_catalogue.groupBy("marque").count()
marques_count_co2 = df_co2.groupBy("marque").count()

marques_count_catalogue = marques_count_catalogue.withColumnRenamed("count", "count_catalogue")
marques_count_co2 = marques_count_co2.withColumnRenamed("count", "count_co2")

marques_count_joined = marques_count_catalogue.join(
    marques_count_co2,
    on="marque",
    how="outer"  # Utilise "left", "right", ou "outer" si nécessaire
).select(
    "marque",
    F.col("count_catalogue"),
    F.col("count_co2")
)

marques_count_joined = marques_count_joined.fillna(0, subset=["count_catalogue", "count_co2"])


marques_count_joined.show(n=1000)

+----------+---------------+---------+
|    marque|count_catalogue|count_co2|
+----------+---------------+---------+
|      audi|             20|        8|
|   bentley|              0|        1|
|       bmw|             20|       12|
|   citroen|              0|        2|
|     dacia|              5|        0|
|  daihatsu|              5|        0|
|        ds|              0|        2|
|      ford|             10|        0|
|     honda|              5|        0|
|   hyundai|              5|        3|
|    jaguar|             10|        1|
|       kia|             15|        6|
|    lancia|             10|        0|
|      land|              0|        5|
|  mercedes|             20|       42|
|      mini|             10|        2|
|mitsubishi|              0|        2|
|    nissan|             15|        9|
|   peugeot|             10|        5|
|   porsche|              0|        4|
|   renault|             40|        3|
|      saab|             10|        0|
|     skoda|             

In [10]:
modele_count_catalogue = df_catalogue.groupBy("modele").count()
modele_count_co2 = df_co2.groupBy("modele").count()

modele_count_catalogue = modele_count_catalogue.withColumnRenamed("count", "count_catalogue")
modele_count_co2 = modele_count_co2.withColumnRenamed("count", "count_co2")

modele_count_joined = modele_count_catalogue.join(
    modele_count_co2,
    on="modele",
    how="outer"  # Utilise "left", "right", ou "outer" si nécessaire
).select(
    "modele",
    F.col("count_catalogue"),
    F.col("count_co2")
)

modele_count_joined_match = modele_count_catalogue.join(
    modele_count_co2,
    on="modele",
    how="inner"  # Utilise "left", "right", ou "outer" si nécessaire
).select(
    "modele",
    F.col("count_catalogue"),
    F.col("count_co2")
)

modele_count_joined = modele_count_joined.fillna(0, subset=["count_catalogue", "count_co2"])


modele_count_joined.show(n=1000, truncate=False)
modele_count_joined_match.show(n=1000, truncate=False)

lignes_filtrees = modele_count_joined.filter(
    (col('count_catalogue') > 0) & (col('count_co2') > 0)
)

nombre_de_lignes = lignes_filtrees.count()

print(f"Nombre de lignes où count_catalogue et count_co2 sont supérieurs à 0 : {nombre_de_lignes}")

+---------------------------------------------------------------------------------------------------+---------------+---------+
|modele                                                                                             |count_catalogue|count_co2|
+---------------------------------------------------------------------------------------------------+---------------+---------+
|1007 1.4                                                                                           |10             |0        |
|120i                                                                                               |10             |0        |
|208 e- tense (136 ch)                                                                              |0              |1        |
|225xe active tourer                                                                                |0              |1        |
|330e berline                                                                                       |0  

### Remarque 003
- Certain **é** sont affiché en ã©
- Aucun modele de **df_co2** ne correspond a un modèle de **df_catalogue**

In [15]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Définir la fonction pour extraire le modèle
def extract_model(modele_str):
    words = modele_str.strip().split()
    model_words = []
    # Liste de mots à arrêter
    stopwords = set([
        'hybrid', 'hybride', 'electric', 'électrique', 'electrique', 'tsi', 'tfsie', 'gdi', 
        'bluehdi', 'tdi', 'fsi', 'e-tron', 'plug-in', 'phev', 'engine', 'twin', 'turbo', 
        'e-tense', 'edrive', 'dci', 'cdti', 'ecoboost', 'ecotec', 'mpi', 'cabrio', 'coupe', 'berline', 'combi' 
    ])
    for word in words:
        word_clean = word.lower().strip('()')
        if word_clean in stopwords:
            break
        elif re.match(r'^\d+\.\d+(ch|kw|t)?$', word_clean):  # nombres décimaux
            break
        elif re.match(r'^\d+(ch|kw|t)$', word_clean):  # nombres avec suffixes
            break
        else:
            model_words.append(word)
    return ' '.join(model_words)

# Créer une UDF (User Defined Function)
extract_model_udf = udf(extract_model, StringType())

# Appliquer la fonction au DataFrame
test = modele_count_joined.withColumn('modele', extract_model_udf(modele_count_joined.modele))

test = test.withColumn('count_catalogue', col('count_catalogue').cast('int'))
test = test.withColumn('count_co2', col('count_co2').cast('int'))
# Agréger les données par 'model'
test = test.groupBy('modele').agg(
    sum('count_catalogue').alias('total_count_catalogue'),
    sum('count_co2').alias('total_count_co2')
)

# Afficher le résultat
test.show(truncate=False)


TypeError: unsupported operand type(s) for +: 'int' and 'str'